<a href="https://colab.research.google.com/github/Iddo77/YOLO_drone_detection/blob/main/YOLO_drones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install scikit-learn

In [1]:
import os
import random
from sklearn.model_selection import train_test_split

In [2]:
# constants
NUM_TRAIN = 300
RANDOM_STATE = 36

In [3]:
# this notebook is expected to run from the Scripts subfolder in the virtual environment
# get a the project_path, which is 2 directories higher (just above the virtual environment)
note_book_path = os.path.abspath('')
project_path = os.path.dirname(os.path.dirname(note_book_path))
print(project_path)
# get a relative path to the dataset folder
dataset_path = os.path.join(project_path, "dataset")
print(dataset_path)

D:\OU Master AI\Research methods\Project\YOLO_drone_detection
D:\OU Master AI\Research methods\Project\YOLO_drone_detection\dataset


In [20]:
os.chdir(project_path)
!git clone https://github.com/WongKinYiu/yolov7.git

Cloning into 'yolov7'...


In [18]:
os.chdir(os.path.join(project_path, 'yolov7'))
!pip install -r requirements.txt

  Using cached matplotlib-3.7.1-cp311-cp311-win_amd64.whl (7.6 MB)
  Using cached opencv_python-4.7.0.72-cp37-abi3-win_amd64.whl (38.2 MB)
  Using cached requests-2.30.0-py3-none-any.whl (62 kB)
  Using cached torch-2.0.0-cp311-cp311-win_amd64.whl (172.3 MB)
  Using cached torchvision-0.15.1-cp311-cp311-win_amd64.whl (1.2 MB)
  Using cached tensorboard-2.13.0-py3-none-any.whl (5.6 MB)
  Using cached pandas-2.0.1-cp311-cp311-win_amd64.whl (10.6 MB)
  Using cached seaborn-0.12.2-py3-none-any.whl (293 kB)
  Using cached thop-0.1.1.post2209072238-py3-none-any.whl (15 kB)
  Using cached contourpy-1.0.7-cp311-cp311-win_amd64.whl (162 kB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached fonttools-4.39.3-py3-none-any.whl (1.0 MB)
  Using cached kiwisolver-1.4.4-cp311-cp311-win_amd64.whl (55 kB)
  Using cached charset_normalizer-3.1.0-cp311-cp311-win_amd64.whl (96 kB)
  Using cached certifi-2023.5.7-py3-none-any.whl (156 kB)
  Using cached filelock-3.12.0-py3-none-any.whl (

In [4]:
def create_train_val_test_txt_files():
    # get all image files and shuffle them
    all_images = [f for f in os.listdir(dataset_path) if f.endswith('.jpg')]
    random.shuffle(all_images)

    # Split the dataset into training, validation, and test sets
    train_images = all_images[:NUM_TRAIN]
    test_val_images = all_images[NUM_TRAIN:]

    # split the test_val set into validation and test sets
    val_images, test_images = train_test_split(test_val_images, 
                                               train_size=0.5, 
                                               test_size=0.5, 
                                               random_state=RANDOM_STATE)

    # Write the paths to the train, val, and test text files
    with open(os.path.join(project_path, 'train.txt'), 'w') as f:
        for img in train_images:
            f.write(os.path.join(dataset_path, img) + '\n')

    with open(os.path.join(project_path, 'val.txt'), 'w') as f:
        for img in val_images:
            f.write(os.path.join(dataset_path, img) + '\n')

    with open(os.path.join(project_path, 'test.txt'), 'w') as f:
        for img in test_images:
            f.write(os.path.join(dataset_path, img) + '\n')

In [5]:
def create_custom_yaml_file():
    """
    Creates a custom YAML file for YOLO that specifies the paths to the training, 
    validation, and test sets.
    """
    yaml_text = f"""train: '{os.path.join(project_path, "train.txt")}'
val: '{os.path.join(project_path, "val.txt")}'
test: '{os.path.join(project_path, "test.txt")}'
nc: 1
names: ['drone']
"""
    with open(os.path.join(project_path, 'custom.yaml'), 'w') as f:
        f.write(yaml_text)


In [6]:
create_train_val_test_txt_files()
create_custom_yaml_file()

In [10]:
# finetune the base model using the custom.yaml file created above
os.chdir(os.path.join(project_path, 'yolov7'))
!python train.py --workers 8 --device 0 --batch-size 16 --data "{os.path.join(project_path, 'custom.yaml')}" --img 640 640 --cfg cfg/training/yolov7.yaml --weights './yolov7_training.pt' --name yolov7-custom --hyp data/hyp.scratch.custom.yaml

Traceback (most recent call last):
  File "D:\OU Master AI\Research methods\Project\YOLO_drone_detection\yolov7\train.py", line 12, in <module>
    import torch.distributed as dist
  File "D:\OU Master AI\Research methods\Project\YOLO_drone_detection\yolo-drones-env\Lib\site-packages\torch\__init__.py", line 122, in <module>
    raise err
OSError: [WinError 1114] A dynamic link library (DLL) initialization routine failed. Error loading "D:\OU Master AI\Research methods\Project\YOLO_drone_detection\yolo-drones-env\Lib\site-packages\torch\lib\nvfuser_codegen.dll" or one of its dependencies.
